In [347]:
# !pip install --upgrade implicit

In [348]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [349]:
data = pd.read_csv('./data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [350]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [351]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [352]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [353]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = items_weights['item_id']
    weights = items_weights['weight']
    
    # передаем дополнительно веса
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

In [354]:
# получить n самых популярных по сумме покупок товаров
def get_popular_by_svalue(data_in, n_popular=0):
    popular = data_in.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    if n_popular > 0:
        popular = popular[:n_popular]
    return popular

# подготовим веса по объему продаж
def get_weighted_polpular(data_in, n_popular=0):
    
    popular = get_popular_by_svalue(data_in,n_popular=n_popular)

    # посолим и прологарифмируем
    popular['weight'] = popular['sales_value'].apply(lambda x: np.log(x+1e-5))
    # приведем мин в 0 (сделаем неотрицательными)
    popular['weight'] = popular['weight'] - popular['weight'].min()  

    log_sum = popular['weight'].sum()
    # найдем веса
    popular['weight'] = popular['weight']/log_sum

    weighted_popular = popular[['item_id', 'weight']].copy()
    print(weighted_popular['weight'].sum())
    return weighted_popular


In [355]:
w_popular = get_weighted_polpular(data_train)

1.0


**тест взвешенных рекомендаций**

In [356]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(w_popular, n=5))
result.head(5)


CPU times: user 4.53 s, sys: 15.3 ms, total: 4.54 s
Wall time: 4.54 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[886686, 976179, 2353902, 9365504, 12949748]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5577580, 1077302, 7046295, 1135096, 1201855]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[953900, 6904531, 9368446, 15926709, 1450899]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1064314, 945905, 949830, 964908, 1072987]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[15660120, 13417845, 935688, 877443, 10454153]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [357]:
pred_basic = pd.read_csv('./data/predictions_basic.csv')
pred_basic.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


**Precision@5**

In [358]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    def _str_to_list(in_str):
        return in_str[1:-1].split(',')

    if type(recommended_list) is type('str'):
        recommended_list = _str_to_list(recommended_list)
        
    if type(bought_list) is type('str'):
        bought_list = _str_to_list(bought_list)

    # непосредственно расчет    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

**функция тестирование датасета**

In [359]:
def test_prediction_at_k(data_pr, actual_idx=1, start_idx=2):
    actual_col = data_pr.columns[actual_idx]
    recomend_algo_cols = data_pr.columns[start_idx:]

    algos_means = []
    for algo in recomend_algo_cols:
        algo_mean = data_pr.apply(lambda row: precision_at_k(row[algo], row[actual_col]), axis=1).mean()
        algos_means.append(algo_mean)
        print(f'Алгоритм: {algo}, mean = {algo_mean}')

    max_idx = np.asarray(algos_means).argmax()
    print()
    print(f'Максимальный результат у {recomend_algo_cols[max_idx]} = {algos_means[max_idx]}')
    print()

In [360]:
%%time
# тестирование предоставленного датасета
test_prediction_at_k(pred_basic)

Алгоритм: random_recommendation, mean = 0.0
Алгоритм: popular_recommendation, mean = 0.0010773751224289912
Алгоритм: itemitem, mean = 0.0
Алгоритм: cosine, mean = 0.0
Алгоритм: tfidf, mean = 0.0
Алгоритм: own_purchases, mean = 0.0

Максимальный результат у popular_recommendation = 0.0010773751224289912

CPU times: user 2.76 s, sys: 96.4 ms, total: 2.86 s
Wall time: 2.69 s


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

### Беизлайны

In [361]:
#освежим результаты
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [362]:
# возьмем 5000 самых популярных товаров
popular = get_popular_by_svalue(data_train, n_popular=5000)
popular.head()

,item_id,sales_value
55470,6534178,447799.94
55430,6533889,40483.34
28895,1029743,35764.66
55465,6534166,30170.77
34707,1082185,26029.96


### 1.1 Random recommendation

In [363]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [364]:
%%time

items = popular.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

CPU times: user 490 ms, sys: 0 ns, total: 490 ms
Wall time: 488 ms


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1062863, 1110042, 1038745, 1120443, 1126703]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[959316, 1044207, 926796, 1136524, 1041259]"


### 1.2 Popularity-based recommendation

In [365]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [366]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(popular, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 13.6 ms, sys: 0 ns, total: 13.6 ms
Wall time: 12.6 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1062863, 1110042, 1038745, 1120443, 1126703]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[959316, 1044207, 926796, 1136524, 1041259]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### 1.3 Weighted random recommender

In [367]:
w_popular = get_weighted_polpular(popular)

1.0


In [368]:
%%time
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(w_popular, n=5))
result.head(2)

CPU times: user 820 ms, sys: 3.65 ms, total: 824 ms
Wall time: 821 ms


,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1062863, 1110042, 1038745, 1120443, 1126703]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8177622, 946977, 1036347, 1099510, 965292]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[959316, 1044207, 926796, 1136524, 1041259]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1065380, 1138443, 12810391, 1047619, 998239]"


In [369]:
%%time
# промежуточное тестирование бейзлайнов
test_prediction_at_k(result)

Алгоритм: random_recommendation, mean = 0.005778648383937317
Алгоритм: popular_recommendation, mean = 0.15523996082272282
Алгоритм: weighted_random_recommendation, mean = 0.011557296767874634

Максимальный результат у popular_recommendation = 0.15523996082272282

CPU times: user 1.32 s, sys: 44.9 ms, total: 1.36 s
Wall time: 1.32 s


# 2. Детерминированные алгоритмы item-item

## 2.1 Item-Item Recommender / ItemKNN

In [370]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [371]:
# popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
# popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

# top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [372]:
top_5000 = popular.item_id.tolist()
K = 7

In [373]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/home/voki/DEV_ML/_env_/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,259120,397896,420647,480014,818981,819063,819255,819304,819308,819330,...,15926844,15926886,15971546,15972074,15972298,15972565,15972790,16053266,16100266,16769635
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [374]:
user_item_matrix.shape

(2499, 5001)

In [375]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

4.809970378873405

In [376]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [377]:
%%time

model = ItemItemRecommender(K=K, num_threads=0) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=True)

CPU times: user 2.43 s, sys: 7.5 ms, total: 2.44 s
Wall time: 1.26 s


In [378]:
recs

[(1988, 56145.0),
 (2132, 26506.0),
 (2567, 5998.0),
 (2140, 4260.0),
 (758, 1775.0)]

In [379]:
[id_to_itemid[rec[0]] for rec in recs]

[981760, 995242, 1029743, 995785, 878996]

In [380]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 196 ms, sys: 0 ns, total: 196 ms
Wall time: 194 ms


In [381]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1062863, 1110042, 1038745, 1120443, 1126703]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8177622, 946977, 1036347, 1099510, 965292]","[981760, 1127831, 1098066, 826249, 878996]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[959316, 1044207, 926796, 1136524, 1041259]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1065380, 1138443, 12810391, 1047619, 998239]","[981760, 995242, 840361, 1029743, 961554]"


### 4.2 Косинусное сходство и CosineRecommender

In [382]:
%%time

model = CosineRecommender(K=K, num_threads=0) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=False)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)

CPU times: user 2.47 s, sys: 2.61 ms, total: 2.47 s
Wall time: 1.29 s


In [383]:
[id_to_itemid[rec[0]] for rec in recs]

[981760, 1127831, 1098066, 826249, 878996]

In [384]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 215 ms, sys: 0 ns, total: 215 ms
Wall time: 213 ms


In [385]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1062863, 1110042, 1038745, 1120443, 1126703]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8177622, 946977, 1036347, 1099510, 965292]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 826249, 878996]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[959316, 1044207, 926796, 1136524, 1041259]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1065380, 1138443, 12810391, 1047619, 998239]","[981760, 995242, 840361, 1029743, 961554]","[981760, 1004906, 961554, 859075, 1096036]"


### 4.3 TF-IDF взвешивание и TFIDFRecommender

Если 2 юзера оба купили очень популярный товар, то это еще не значит,что они похожи   
Если 2 юзера оба купили редкий товар, то они похожи

Занижаем вес популярных товаров при расчете расстояний между пользователями

In [386]:
%%time

model = TFIDFRecommender(K=K, num_threads=0) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=False)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)

CPU times: user 2.47 s, sys: 3.5 ms, total: 2.48 s
Wall time: 1.3 s


In [387]:
[id_to_itemid[rec[0]] for rec in recs]

[981760, 1127831, 1098066, 826249, 878996]

In [388]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=False)])

CPU times: user 226 ms, sys: 29 µs, total: 226 ms
Wall time: 224 ms


In [389]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1062863, 1110042, 1038745, 1120443, 1126703]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8177622, 946977, 1036347, 1099510, 965292]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 826249, 878996]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[959316, 1044207, 926796, 1136524, 1041259]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1065380, 1138443, 12810391, 1047619, 998239]","[981760, 995242, 840361, 1029743, 961554]","[981760, 1004906, 961554, 859075, 1096036]","[981760, 1004906, 961554, 840361, 859075]"


### 4.4 Трюк

In [390]:
%%time
# здесь оставляем К = 1
model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=False)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)

CPU times: user 2.38 s, sys: 0 ns, total: 2.38 s
Wall time: 1.24 s


In [391]:
[id_to_itemid[rec[0]] for rec in recs]

[12172240, 12384953, 12810393, 15926844, 12672980]

In [392]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])

CPU times: user 170 ms, sys: 44 µs, total: 170 ms
Wall time: 167 ms


In [393]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1062863, 1110042, 1038745, 1120443, 1126703]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8177622, 946977, 1036347, 1099510, 965292]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[959316, 1044207, 926796, 1136524, 1041259]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1065380, 1138443, 12810391, 1047619, 998239]","[981760, 995242, 840361, 1029743, 961554]","[981760, 1004906, 961554, 859075, 1096036]","[981760, 1004906, 961554, 840361, 859075]","[999999, 1082185, 1098066, 6534178, 1127831]"


### 4.5 Измерим качество по precision@5

In [394]:
%%time
test_prediction_at_k(result)

Алгоритм: random_recommendation, mean = 0.005778648383937317
Алгоритм: popular_recommendation, mean = 0.15523996082272282
Алгоритм: weighted_random_recommendation, mean = 0.011557296767874634
Алгоритм: itemitem, mean = 0.030754162585700295
Алгоритм: cosine, mean = 0.036141038197845254
Алгоритм: tfidf, mean = 0.03643486777668952
Алгоритм: own_purchases, mean = 0.17724453150506042

Максимальный результат у own_purchases = 0.17724453150506042

CPU times: user 3.07 s, sys: 89 ms, total: 3.16 s
Wall time: 3.08 s
